In [1]:
import glob
import json
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import re
import operator 
from collections import Counter
from nltk import bigrams
import pandas as pd
import csv
import unicodecsv
from os import listdir
from os.path import isfile, join
import glob
import wordcloud
import matplotlib.pyplot as plt

In [2]:
#reding dta
df_price = pd.read_csv('full_DataFrame/combined_data/price/minute_data_till_05_Mar_2018_02_47_00.csv')
df_price.head()

,Unnamed: 0,close,high,low,open,time,volumefrom,volumeto
0,1,11513.76,11523.56,11513.55,11522.91,1520218020,99.40,1142729.05
1,2,11522.44,11526.79,11522.44,11526.79,1520217960,36.48,420428.52
2,3,11526.79,11530.34,11526.77,11527.09,1520217900,87.55,1009687.20
3,4,11527.09,11529.46,11526.33,11529.20,1520217840,109.72,1264631.41
4,5,11529.22,11529.22,11521.30,11521.30,1520217780,222.04,2555000.41


In [3]:
#converting time into pandas date time
df_price['time'] = pd.to_datetime(df_price['time'],unit='s')
df_price.head()

,Unnamed: 0,close,high,low,open,time,volumefrom,volumeto
0,1,11513.76,11523.56,11513.55,11522.91,2018-03-05 02:47:00,99.40,1142729.05
1,2,11522.44,11526.79,11522.44,11526.79,2018-03-05 02:46:00,36.48,420428.52
2,3,11526.79,11530.34,11526.77,11527.09,2018-03-05 02:45:00,87.55,1009687.20
3,4,11527.09,11529.46,11526.33,11529.20,2018-03-05 02:44:00,109.72,1264631.41
4,5,11529.22,11529.22,11521.30,11521.30,2018-03-05 02:43:00,222.04,2555000.41


In [4]:
print(df_price.time.min())
print(df_price.time.max())

2018-02-26 02:56:00
2018-03-05 02:47:00


In [5]:
mask = (df_price['time'] > '2018-02-27 05:05:02') & (df_price['time'] <= '2018-03-03 08:43:41')

In [6]:
masked = df_price.loc[mask]

In [7]:
masked.head()

,Unnamed: 0,close,high,low,open,time,volumefrom,volumeto
2524,2525,11329.51,11330.38,11324.16,11324.48,2018-03-03 08:43:00,19.54,221634.51
2525,2526,11324.48,11325.97,11324.04,11325.97,2018-03-03 08:42:00,15.44,175207.51
2526,2527,11326.66,11328.67,11325.27,11327.40,2018-03-03 08:41:00,33.73,387171.09
2527,2528,11327.40,11330.56,11326.67,11329.07,2018-03-03 08:40:00,35.22,406279.13
2528,2529,11330.22,11332.91,11330.18,11331.26,2018-03-03 08:39:00,37.17,423019.24


In [8]:
masked.shape

(5978, 8)

In [9]:
masked.to_msgpack("full_DataFrame/combined_data/master/data_price_min.msg")

In [10]:
masked.set_index('time', inplace=True)

In [11]:
masked.head()

,Unnamed: 0,close,high,low,open,volumefrom,volumeto
time,,,,,,,
2018-03-03 08:43:00,2525,11329.51,11330.38,11324.16,11324.48,19.54,221634.51
2018-03-03 08:42:00,2526,11324.48,11325.97,11324.04,11325.97,15.44,175207.51
2018-03-03 08:41:00,2527,11326.66,11328.67,11325.27,11327.40,33.73,387171.09
2018-03-03 08:40:00,2528,11327.40,11330.56,11326.67,11329.07,35.22,406279.13
2018-03-03 08:39:00,2529,11330.22,11332.91,11330.18,11331.26,37.17,423019.24


In [12]:
data_2min = masked.groupby(pd.TimeGrouper('2Min')).mean()

In [13]:
data_2min.head()

,Unnamed: 0,close,high,low,open,volumefrom,volumeto
time,,,,,,,
2018-02-27 05:06:00,8501.5,10302.905,10305.870,10301.415,10305.450,23.955,246606.365
2018-02-27 05:08:00,8499.5,10280.955,10292.110,10280.085,10291.670,42.220,434544.860
2018-02-27 05:10:00,8497.5,10263.575,10280.775,10262.600,10280.160,123.155,1265029.010
2018-02-27 05:12:00,8495.5,10255.455,10257.025,10245.155,10250.020,69.010,706970.855
2018-02-27 05:14:00,8493.5,10258.500,10261.020,10250.295,10255.775,77.405,806866.155


In [14]:
data_2min.shape

(2989, 7)

In [15]:
data_5min = masked.groupby(pd.TimeGrouper('5Min')).mean()
data_5min.head()

,Unnamed: 0,close,high,low,open,volumefrom,volumeto
time,,,,,,,
2018-02-27 05:05:00,8500.5,10291.930,10298.990,10290.750,10298.560,33.0875,340575.6125
2018-02-27 05:10:00,8496.0,10258.446,10266.906,10253.130,10263.548,83.3880,856800.9140
2018-02-27 05:15:00,8491.0,10265.348,10268.046,10260.372,10263.234,38.4800,399515.1720
2018-02-27 05:20:00,8486.0,10280.298,10281.710,10274.240,10276.078,15.6800,161987.9040
2018-02-27 05:25:00,8481.0,10275.134,10279.566,10273.276,10279.124,15.6360,161420.9880


In [16]:
data_5min.shape

(1196, 7)

In [17]:
data_2min_volume = []
for i in range(0, len(data_2min)):
    data_2min_volume.append(data_2min['volumeto'][i]-data_2min['volumefrom'][i])
data_2min_volume_df = pd.DataFrame(data_2min_volume, columns =[['volume']])
data_2min_volume_df.head()

,volume
0,246582.410
1,434502.640
2,1264905.855
3,706901.845
4,806788.750


In [18]:
data_2min_volume_df.shape

(2989, 1)

In [19]:
data_2min = data_2min.reset_index()
data_2min = data_2min.drop(['Unnamed: 0'], axis=1)

In [20]:
data_2min.head()

,time,close,high,low,open,volumefrom,volumeto
0,2018-02-27 05:06:00,10302.905,10305.870,10301.415,10305.450,23.955,246606.365
1,2018-02-27 05:08:00,10280.955,10292.110,10280.085,10291.670,42.220,434544.860
2,2018-02-27 05:10:00,10263.575,10280.775,10262.600,10280.160,123.155,1265029.010
3,2018-02-27 05:12:00,10255.455,10257.025,10245.155,10250.020,69.010,706970.855
4,2018-02-27 05:14:00,10258.500,10261.020,10250.295,10255.775,77.405,806866.155


In [21]:
dat1 = data_2min_volume_df[['volume']]
data_2min = data_2min.join(dat1)
data_2min.head()

,time,close,high,low,open,volumefrom,volumeto,volume
0,2018-02-27 05:06:00,10302.905,10305.870,10301.415,10305.450,23.955,246606.365,246582.410
1,2018-02-27 05:08:00,10280.955,10292.110,10280.085,10291.670,42.220,434544.860,434502.640
2,2018-02-27 05:10:00,10263.575,10280.775,10262.600,10280.160,123.155,1265029.010,1264905.855
3,2018-02-27 05:12:00,10255.455,10257.025,10245.155,10250.020,69.010,706970.855,706901.845
4,2018-02-27 05:14:00,10258.500,10261.020,10250.295,10255.775,77.405,806866.155,806788.750


In [22]:
data_2min.shape

(2989, 8)

In [23]:
data_2min = data_2min[['time', 'close', 'high', 'low', 'volume']]

In [24]:
data_2min.head()

,time,close,high,low,volume
0,2018-02-27 05:06:00,10302.905,10305.870,10301.415,246582.410
1,2018-02-27 05:08:00,10280.955,10292.110,10280.085,434502.640
2,2018-02-27 05:10:00,10263.575,10280.775,10262.600,1264905.855
3,2018-02-27 05:12:00,10255.455,10257.025,10245.155,706901.845
4,2018-02-27 05:14:00,10258.500,10261.020,10250.295,806788.750


In [25]:
data_2min.to_msgpack("full_DataFrame/combined_data/master/price_data_2min.msg")

In [26]:
data_5min_volume = []
for i in range(0, len(data_5min)):
    data_5min_volume.append(data_5min['volumeto'][i]-data_5min['volumefrom'][i])
data_5min_volume_df = pd.DataFrame(data_5min_volume, columns =[['volume']])
data_5min_volume_df.head()

,volume
0,340542.525
1,856717.526
2,399476.692
3,161972.224
4,161405.352


In [27]:
data_5min_volume_df.shape

(1196, 1)

In [28]:
data_5min = data_5min.reset_index()
data_5min = data_5min.drop(['Unnamed: 0'], axis=1)
data_5min.head()

,time,close,high,low,open,volumefrom,volumeto
0,2018-02-27 05:05:00,10291.930,10298.990,10290.750,10298.560,33.0875,340575.6125
1,2018-02-27 05:10:00,10258.446,10266.906,10253.130,10263.548,83.3880,856800.9140
2,2018-02-27 05:15:00,10265.348,10268.046,10260.372,10263.234,38.4800,399515.1720
3,2018-02-27 05:20:00,10280.298,10281.710,10274.240,10276.078,15.6800,161987.9040
4,2018-02-27 05:25:00,10275.134,10279.566,10273.276,10279.124,15.6360,161420.9880


In [29]:
dat1 = data_5min_volume_df[['volume']]
data_5min = data_5min.join(dat1)
data_5min.head()

,time,close,high,low,open,volumefrom,volumeto,volume
0,2018-02-27 05:05:00,10291.930,10298.990,10290.750,10298.560,33.0875,340575.6125,340542.525
1,2018-02-27 05:10:00,10258.446,10266.906,10253.130,10263.548,83.3880,856800.9140,856717.526
2,2018-02-27 05:15:00,10265.348,10268.046,10260.372,10263.234,38.4800,399515.1720,399476.692
3,2018-02-27 05:20:00,10280.298,10281.710,10274.240,10276.078,15.6800,161987.9040,161972.224
4,2018-02-27 05:25:00,10275.134,10279.566,10273.276,10279.124,15.6360,161420.9880,161405.352


In [30]:
data_5min = data_5min[['time', 'close', 'high', 'low', 'volume']]

In [31]:
data_5min.head()

,time,close,high,low,volume
0,2018-02-27 05:05:00,10291.930,10298.990,10290.750,340542.525
1,2018-02-27 05:10:00,10258.446,10266.906,10253.130,856717.526
2,2018-02-27 05:15:00,10265.348,10268.046,10260.372,399476.692
3,2018-02-27 05:20:00,10280.298,10281.710,10274.240,161972.224
4,2018-02-27 05:25:00,10275.134,10279.566,10273.276,161405.352


In [32]:
data_5min.to_msgpack("full_DataFrame/combined_data/master/price_data_5min.msg")